---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2019</h1>
<h1 style="font-size: 250%;">Problem Set #4, Version 2.0</h1>
<h1>Issued Tuesday 4/9/2019; Due Tuesday, 4/23/2019</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

---
**Problem #1:** Of the 10 sequences provided [*here*](https://csbio.unc.edu/mcmillan/Comp555S19/data/PS4.fa), find the 3 sequences that maximize the sum of their Pairwise Global Alignments. Assume a weight of 1 for all *matches* and -1 for all *mismatches and indels* in your alignment. (Note: you only need to find the best score here, not the alignment, thus you do not need the backtracking matrix).

Report the *index* of the 3 sequences and the three alignment scores between them as a comma separated list with three rows of three numbers in the following format:

        seqIndexA, seqIndexB, AB_AlignmentScore
        seqIndexA, seqIndexC, AC_AlignmentScore
        seqIndexB, seqIndexC, BC_AlignmentScore

In [ ]:
import numpy

def GlobalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s)

In [ ]:
data = open("data/PS4.fa","r").read().split("\n")
match = {('A','A'):  1, ('A','C'): -1, ('A','G'): -1, ('A','T'): -1,
         ('C','A'): -1, ('C','C'):  1, ('C','G'): -1, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -1, ('G','G'):  1, ('G','T'): -1,
         ('T','A'): -1, ('T','C'): -1, ('T','G'): -1, ('T','T'):  1}

In [ ]:
N = len(data)
print("N=",N)
score = dict()
for i in range(N):
    for j in range(i+1, N):
        v = data[i][:]
        w = data[j][:]
        s= GlobalAlign(v,w,match,-1)
        score[i,j] = s[-1,-1]
        print(i, j,"Best score =", s[-1,-1]) #Extremely Slow, takes forever

---
**Problem #2:** The sequences from problem one are from endogenous retroviruses (ERVs). ERVs have a specific genomic structure, where they begin and end with a common highly conserved long terminal repeat (LTR) sequence. Using the 3 sequences you found in Problem #1, attempt to find the LTR sequences by performing a Local Alignment of the first and last 1000 bases from the same sequence. Repeat this for all 3 sequences using the same scoring matrix as in part one. Report the respective start and end indices from each of the three Local Alignments. Here you will need the backtracking matrix. Your answer should be 3 rows of 5 comma separated numbers of the form:

       seqIndexA, startOfProximalLTR, endOfProximalLTR, startOfDistalLTR, endOfDistalLTR
       seqIndexB, startOfProximalLTR, endOfProximalLTR, startOfDistalLTR, endOfDistalLTR
       seqIndexC, startOfProximalLTR, endOfProximalLTR, startOfDistalLTR, endOfDistalLTR

In [9]:
data = open("data/PS4.fa","r").read().split("\n")
match = {('A','A'):  1, ('A','C'): -1, ('A','G'): -1, ('A','T'): -1,
         ('C','A'): -1, ('C','C'):  1, ('C','G'): -1, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -1, ('G','G'):  1, ('G','T'): -1,
         ('T','A'): -1, ('T','C'): -1, ('T','G'): -1, ('T','T'):  1}

In [22]:
def LocalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = max(s[i-1,j] + indel, 0)
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = max(s[i,j-1] + indel, 0)
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score, 0)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            elif (s[i,j] == score):
                b[i,j] = 3
            else:
                b[i,j] = 0
    return (s, b)

def LocalIndexAlignment(s,b,v,w,i,j):
    if (s[i,j] == 0): 
        return i, j
    if (b[i,j] == 3):
        return LocalIndexAlignment(s,b,v,w,i-1,j-1)
    if (b[i,j] == 2):
        return LocalIndexAlignment(s,b,v,w,i,j-1)
    if (b[i,j] == 1):
        return LocalIndexAlignment(s,b,v,w,i-1,j)
        
for i, seq in enumerate(data[:3]):
    s, b = LocalAlign(seq[:1000], seq[-1000:], match, -1)
    eP, eD = numpy.unravel_index(numpy.argmax(s, axis=None), s.shape)
    sP, sD = LocalIndexAlignment(s, b, seq[:1000], seq[-1000:], eP, eD)
    
    print(i,",", sP,",", eP,",", len(seq) - 1000 + sD, ",", len(seq) - 1000 + eD)

0 , 0 , 319 , 7158 , 7477
1 , 0 , 319 , 7158 , 7477
2 , 0 , 319 , 7160 , 7479


---
**Problem #3:** Using the three LTRs alignments that you found in Problem #2, report a *consensus LTR sequence*.

In [12]:
def Consensus(s, DNA, k):
    """ compute the consensus k-Motif of an alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore, DNA = list of nucleotide strings,
            k = Target Motif length """
    consensus = ''
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("ACGT",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        consensus += max(cnt.items(), key=lambda tup: tup[1])[0]
    return consensus

DNA = [data[0], data[0], data[1], data[1], data[2], data[2]]
s = [0, 7158, 0, 7158, 0, 7160]

Consensus(s, DNA, 319)

'TGTTGCGGCCGCCAGCAGCTCGCAACGTGAACGGTTCGACTGAGAAGGCCGCTCGAGCTGTAAGAGAGGAATCTAGACGGGGCAAAAGAAGAAACGGAGCTAAGGCAATTTCATACTGATCAAAGCTCAAATTTTATTGTTGCGACACCAGTTATGAAGGAAGGGGGAGGGGACCCGATTCCCGCCGAATAATCTCTGGTCCAGTAGAAAGGTGCACGGGTGTGGCTCCGCAGGTTCCAGCAGTGGGCGTGGCAGAACGAATGAGCAGGAAGCTCCACCCCGAGCAAGCAGGTTTCAGGCTGGGGGAGGGGAGACTACA'

---
**Problem #4:** Using the consensus subsequence you found in Problem #3, find a *Local Pairwise Alignment* of this consensus LTR to each of the first, and last, 1000 base pairs of the 10 original sequences and report the score for each. This should be a comma separated list of ten rows each with three numbers the following format:

         1, ProximalLTRscore, DistalLTRscore
         2, ProximalLTRscore, DistalLTRscore
                      .
                      .
                      .
         9, ProximalLTRscore, DistalLTRscore
         10, ProximalLTRscore, DistalLTRscore


In [13]:
consensus = data[0][0:319] # possible consensus given from 3

In [14]:
def LocalAlign(v, w):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = max(s[i-1,j]-1, 0)
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = max(s[i,j-1]-1, 0)
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + (1 if v[i-1]==w[j-1] else -1)
            vskip = s[i-1,j]-1
            wskip = s[i,j-1]-1
            s[i,j] = max(vskip, wskip, score, 0)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            elif (s[i,j] == score):
                b[i,j] = 3
            else:
                b[i,j] = 0
    return (s, b)

In [15]:
def FindMaxScore(s):
    maxScore = 0
    maxIndex = [0,0]
    for i in range(s.shape[0]):
        for j in range(s.shape[1]):
            if(s[i,j] > maxScore):
                maxScore = s[i,j]
                maxIndex = [i,j];
    return [maxScore,maxIndex]

In [18]:
length = 1000
for i in range(len(data)):
    s1,b = LocalAlign(data[i][:length],consensus)
    s2,b = LocalAlign(data[i][-length:],consensus)
    
    print(i+1, ",", FindMaxScore(s1)[0],",", FindMaxScore(s2)[0])
print(s1)

1 , 319 , 319
2 , 315 , 315
3 , 317 , 317
4 , 294 , 294
5 , 293 , 291
6 , 282 , 284
7 , 301 , 299
8 , 293 , 295
9 , 291 , 293
10 , 319 , 317


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.